<a href="https://colab.research.google.com/github/cdbolivarz/Accidentality_Predictions_Medellin/blob/master/Limpieza__y_formato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza y formato de datos
En este documento, se realiza el preprocesamiento de los datos obtenidos de los documentos de incidentes georeferenciados.

Librerias

In [1]:
import pandas as pd
import glob
import numpy as np
import re

### Lectura de datos

In [2]:
all_files = glob.glob("dataset/I*.csv")

li = []

for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
    li.append(frame)

df = pd.concat(li, axis=0, ignore_index=True)
df = df.sort_values(by=["FECHA"])
df.tail()

,X,Y,OBJECTID,RADICADO,FECHA,HORA,DIA,PERIODO,CLASE,DIRECCION,DIRECCION_ENC,CBML,TIPO_GEOCOD,GRAVEDAD,BARRIO,COMUNA,DISENO,DIA_NOMBRE,MES,MES_NOMBRE,X_MAGNAMED,Y_MAGNAMED,LONGITUD,LATITUD
86230,-75.572551,6.265264,682514,1.6551e+06,2018/12/31 00:00:00+00,05:10:00,31,2018,Caida Ocupante,CR 64 CL 67,CR 064 067 000 00000,0517,Malla vial,HERIDO,Caribe,Castilla,Tramo de via,LUNES,12,NaN,834530.37,1184805.04,-75.572551,6.265264
86231,-75.703816,6.221806,682515,1.6551e+06,2018/12/31 00:00:00+00,05:40:00,31,2018,Otro,CR 63 CL 91,CR 063 091 000 00000,NaN,No Ubicada,HERIDO,NaN,NaN,Tramo de via,LUNES,12,NaN,820000.00,1180000.00,-75.703816,6.221806
86232,-75.606045,6.258454,682516,1.65542e+06,2018/12/31 00:00:00+00,06:05:00,31,2018,Choque,CL 47 F CR 89 A,CL 047 F 089 A 000 00000,1206,Malla vial,SOLO DAÑOS,Santa Lucía,La América,Interseccion,LUNES,12,NaN,830822.95,1184052.00,-75.606045,6.258454
86234,-75.575120,6.301392,682518,1.6551e+06,2018/12/31 00:00:00+00,07:30:00,31,2018,Caida Ocupante,CL 103 CR 76,CL 103 076 000 00000,0604,Malla vial,HERIDO,Pedregal,Doce de Octubre,Tramo de via,LUNES,12,NaN,834246.07,1188801.36,-75.575120,6.301392
86301,-75.550346,6.273010,682585,1.65511e+06,2018/12/31 00:00:00+00,01:40:00,31,2018,Choque,CR 41 A CL 81,CR 041 A 081 000 00000,0303,Malla vial,HERIDO,Campo Valdés No. 2,Manrique,Interseccion,LUNES,12,NaN,836988.12,1185661.87,-75.550346,6.273010


Que columnas tienen valores nulos

In [ ]:
df.isna().any()

X                False
Y                False
OBJECTID         False
RADICADO         False
FECHA            False
HORA             False
DIA              False
PERIODO          False
CLASE             True
DIRECCION        False
DIRECCION_ENC     True
CBML              True
TIPO_GEOCOD      False
GRAVEDAD         False
BARRIO            True
COMUNA            True
DISENO            True
DIA_NOMBRE       False
MES              False
MES_NOMBRE        True
X_MAGNAMED       False
Y_MAGNAMED       False
LONGITUD         False
LATITUD          False
dtype: bool

Formato datos

In [3]:
#Eliminar filas con NA y columnas que no consideramos importantes para el modelo.
#Asignar mismos nombres a los tipos de accidentes (CLASE)
def format(df):
  df = df.drop(['X', 'Y', 'OBJECTID', 'RADICADO', 'DIRECCION', 'DIRECCION_ENC', 'FECHA',
              'CBML', 'TIPO_GEOCOD', 'DISENO', 'MES_NOMBRE', 'X_MAGNAMED',
              'Y_MAGNAMED'], axis=1)
  df = df.dropna(subset=['CLASE', 'BARRIO', 'COMUNA'])

  df["CLASE"].replace({"Caída de Ocupante": "Caida Ocupante",
                       "Caida de Ocupante": "Caida Ocupante",
                       "Caída Ocupante": "Caida Ocupante",
                       "Choque ": "Choque"}, inplace=True)

  return df

df = format(df)
df.columns = df.columns.str.lower()
df.head()

,hora,dia,periodo,clase,gravedad,barrio,comuna,dia_nombre,mes,longitud,latitud
136790,04:20:00,1,2014,Choque,HERIDO,Campo Amor,Guayabal,MIÉRCOLES,1,-75.579329,6.209160
136789,09:10:00,1,2014,Choque,HERIDO,Barrio Colón,La Candelaria,MIÉRCOLES,1,-75.565164,6.242722
136788,08:50:00,1,2014,Choque,HERIDO,San Miguel,Villa Hermosa,MIÉRCOLES,1,-75.555400,6.252163
136787,08:00:00,1,2014,Otro,HERIDO,La Candelaria,La Candelaria,MIÉRCOLES,1,-75.564683,6.249282
136786,10:00:00,1,2014,Choque,HERIDO,La Milagrosa,Buenos Aires,MIÉRCOLES,1,-75.557573,6.235498


Filas con errores ortograficos en algunas columnas

In [ ]:
df['COMUNA'].drop_duplicates()

In [ ]:
df['BARRIO'].drop_duplicates()

In [ ]:
df.head()

,HORA,DIA,PERIODO,CLASE,GRAVEDAD,BARRIO,COMUNA,DIA_NOMBRE,LONGITUD,LATITUD
0,02:20:00,1,2014,Choque,HERIDO,Loma de los Bernal,Belén,MIÉRCOLES,-75.602726,6.219016
62,04:20:00,1,2014,Choque,HERIDO,Campo Amor,Guayabal,MIÉRCOLES,-75.579329,6.209160
61,09:10:00,1,2014,Choque,HERIDO,Barrio Colón,La Candelaria,MIÉRCOLES,-75.565164,6.242722
60,08:50:00,1,2014,Choque,HERIDO,San Miguel,Villa Hermosa,MIÉRCOLES,-75.555400,6.252163
59,08:00:00,1,2014,Otro,HERIDO,La Candelaria,La Candelaria,MIÉRCOLES,-75.564683,6.249282


In [6]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
len(df['BARRIO'].drop_duplicates())

340

Todos las columnas de *clase, gravedad, barrio, comuna, dia_nombre* se pasaron a miniuscula para evitar posibles inconveniesntes futuros

In [16]:
dfcopy = df.copy()
dfcopy['clase'] = dfcopy['clase'].str.lower()
dfcopy['gravedad'] = dfcopy['gravedad'].str.lower()
dfcopy['barrio'] = dfcopy['barrio'].str.lower()
dfcopy['comuna'] = dfcopy['comuna'].str.lower()
dfcopy['dia_nombre'] = dfcopy['dia_nombre'].str.lower()

## Barrios


Se eliminan las filas donde el barrio sale como un número

In [17]:
for barrio in dfcopy['barrio']:
    try:
        int(barrio)
        dfcopy = dfcopy.replace({'barrio': barrio}, np.nan)
    except:
        pass
dfcopy= dfcopy.dropna(subset=['barrio'])

Para quitar corregir los barrios donde salía **no. #**  para que salga **no.#**

In [18]:
def error_no(row):
    barrio = row['barrio'].strip()
    b = bool(re.search(r'(no)\. \d', barrio))
    if (b):
        b = barrio.split()
        barrio = ' '.join(b[0:-2])
        barrio += ' '+b[-2]+b[-1]
    row['barrio'] = barrio
    return row
dfcopy = dfcopy.apply(error_no,axis=1)

In [ ]:
# Barrios con problemas
# en Medellin se supone hay 275 barrio (segun wikipedia)

# hay varios suburbanos que me parecen raros
# el barrio 'u.p.b'
# creo que todos los barrios que se llaman universidad algo estan mal (el de falcultad de minas existe)


In [19]:
dfcopy = dfcopy.replace({'barrio':{'barrio de jesús':'barrios de jesús',
                                   'berlin':'berlín',
                                   'inst':np.nan,
                                   'loma de los bernal':'la loma de los bernal',
                                   'villa liliam':'villa lilliam'}})

In [8]:
barrios = dfcopy['barrio'].sort_values().unique()
# barrios[200:-1]

In [9]:
len(barrios)

317

## Comunas

In [ ]:
# 1	Comuna 1 - Popular
# 2	Comuna 2 - Santa Cruz
# 3	Comuna 3 - Manrique
# 4	Comuna 4 - Aranjuez
# 5	Comuna 5 - Castilla
# 6	Comuna 6 - Doce de Octubre
# 7	Comuna 7 - Robledo
# 8	Comuna 8 - Villa Hermosa
# 9	Comuna 9 - Buenos Aires
# 10	Comuna 10 - La Candelaria
# 11	Comuna 11 - Laureles-Estadio
# 12	Comuna 12 - La América
# 13	Comuna 13 - San Javier
# 14	Comuna 14 - El Poblado
# 15	Comuna 15 - Guayabal
# 16	Comuna 16 - Belén

In [20]:
dfcopy = dfcopy.replace({'comuna':{ 'alejandro echavarría': 'buenos aires',
                                    'alfonso lópez':'castilla',
                                    'altavista':'belén',
                                    'andalucía':'santa cruz',
                                    'antonio nariño':'san javier',
                                    'au':np.nan,
                                    'barrio colón':'la candelaria',
                                    'bolivariana':'laureles-estadio',
                                    'boston':'la candelaria',
                                    'cabecera san antonio de prado':'corregimiento san antorio de prado',
                                    'calasanz':'la américa',
                                    'calle nueva':'la candelaria',
                                    'campo amor':'guayabal',
                                    'campo valdés no. 1':'aranjuez',
                                    'campo valdés no. 2':'manrique',
                                    'caribe':'castilla',
                                    'cerro nutibara':'belén',
                                    'corazón de jesús':'la candelaria',
                                    'cristo rey':'guayabal',
                                    'el chagualo':'la candelaria',
                                    'el nogal-los almendros':'belén',
                                    'el raizal':'manrique',
                                    'estación villa':'la candelaria',
                                    'florida nueva':'laureles-estadio',
                                    'fátima':'belén',
                                    'girardot':'castilla',
                                    'guayaquil':'la candelaria',
                                    'héctor abad gómez':'castilla'}})

In [21]:
dfcopy = dfcopy.replace({'comuna':{'in':np.nan, 
                                'jesús nazareno':'la candelaria',
                                'la alpujarra':np.nan,
                                'la colina':'guayabal',
                                'la floresta':'la américa',
                                'la rosa':'santa cruz',
                                'las esmeraldas':'aranjuez',
                                'las granjas':'manrique',
                                'las palmas':'la candelaria',
                                'las playas':'belén',
                                'laureles':'laureles-estadio',
                                'laureles estadio':'laureles-estadio', 
                                'los conquistadores':'laureles-estadio',
                                'los mangos':'villa hermosa',
                                'manila':'el poblado',
                                'miranda':'aranjuez',
                                'moravia':'aranjuez',
                                'naranjal':'laureles-estadio',
                                'oleoducto':np.nan,
                                'parque juan pablo ii':'buenos aires',
                                'patio bonito':'el poblado',
                                'perpetuo socorro':'la candelaria',
                                'rosales':'belén',
                                'san pedro':'aranjuez',
                                'santa fé':'guayabal',
                                'santa inés':'manrique',
                                'santa maría de los ángeles':'el poblado',
                                'simón bolívar':'la américa',
                                'sn':np.nan,
                                'suramericana':'laureles-estadio',
                                'toscana':'castilla',
                                'universidad de antioquia':np.nan,
                                'veinte de julio':'san javier',
                                'villa carlota':'el poblado',
                                'villa guadalupe':'popular'}})

In [22]:
comunas = dfcopy['comuna'].sort_values().unique()
comunas

array(['aranjuez', 'belén', 'buenos aires', 'castilla',
       'corregimiento de altavista',
       'corregimiento de san antonio de prado',
       'corregimiento de san cristóbal',
       'corregimiento de san sebastián de palmitas',
       'corregimiento de santa elena',
       'corregimiento san antorio de prado', 'doce de octubre',
       'el poblado', 'guayabal', 'la américa', 'la candelaria',
       'laureles-estadio', 'manrique', 'popular', 'robledo', 'san javier',
       'santa cruz', 'villa hermosa', nan], dtype=object)